In [1]:
import torch
from torchtext.legacy import data
import torch.nn as nn
from torchtext.legacy.data import Field, LabelField, TabularDataset, Dataset

import torch.optim as optim

import time
import pandas as pd
import re
import spacy
from nltk.stem import PorterStemmer

from spacy.lang.en.stop_words import STOP_WORDS

In [2]:
import os
os.getcwd()

'C:\\Users\\Sunny\\Desktop\\Master\\Sem3\\5212\\A1\\Dataset_Assignment1'

In [3]:
# Set up random seed

SEED = 1

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [4]:
# Set up field & loading datasets

spacy_en = spacy.load("en_core_web_sm")

def custom_tokenizer(text):
    return [tok.text for tok in spacy_en.tokenizer(text) if tok.text not in STOP_WORDS]

TEXT = data.Field(sequential=True, tokenize=custom_tokenizer, lower=False)
LABEL = LabelField(dtype = torch.float)
 
train_datafield = [("title", TEXT), 
                   ("abstract", None),
                   ("InformationTheory", LABEL), 
                   ("ComputationalLinguistics", LABEL),
                   ("ComputerVision", LABEL)
                   ]

train_data, test_data = TabularDataset.splits(
    path = "./",
    train = "train.csv", test = "test.csv", format = "csv",
    skip_header = True, fields = train_datafield)

from torchtext.legacy.data import Dataset

def split_dataset(dataset, split_index):
    fields = dataset.fields
    examples = dataset.examples
    top_examples = examples[:split_index]
    remaining_examples = examples[split_index:]
    
    top_dataset = Dataset(top_examples, fields)
    remaining_dataset = Dataset(remaining_examples, fields)
    
    return top_dataset, remaining_dataset

train_data_1000, valid_data = split_dataset(train_data, 1000)

In [5]:
# Building vocab
MAX_VOCAB_SIZE = 1000

TEXT.build_vocab(train_data_1000, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data_1000)

In [37]:
!pip3 install torch==1.9.1+cu111 torchvision==0.10.1+cu111 torchaudio==0.9.1 -f https://download.pytorch.org/whl/torch_stable.html


Looking in links: https://download.pytorch.org/whl/torch_stable.html

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\Sunny\\anaconda3\\Lib\\site-packages\\~=rch\\lib\\asmjit.dll'
Consider using the `--user` option or check the permissions.




     ---------------------------------------- 3.1/3.1 GB 350.4 kB/s eta 0:00:00
     ---------------------------------------- 2.5/2.5 MB 2.1 MB/s eta 0:00:00
     -------------------------------------- 216.0/216.0 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0
    Uninstalling torch-1.9.0:
      Successfully uninstalled torch-1.9.0


In [ ]:
!pip3 install torchtext==0.10.0 

[autoreload of torch.overrides failed: Traceback (most recent call last):
  File "C:\Users\Sunny\anaconda3\lib\site-packages\IPython\extensions\autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "C:\Users\Sunny\anaconda3\lib\site-packages\IPython\extensions\autoreload.py", line 394, in superreload
    module = reload(module)
  File "C:\Users\Sunny\anaconda3\lib\imp.py", line 314, in reload
    return importlib.reload(module)
  File "C:\Users\Sunny\anaconda3\lib\importlib\__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 613, in _exec
  File "<frozen importlib._bootstrap_external>", line 850, in exec_module
  File "<frozen importlib._bootstrap>", line 228, in _call_with_frames_removed
  File "C:\Users\Sunny\anaconda3\lib\site-packages\torch\overrides.py", line 1262, in <module>
    has_torch_function = _add_docstr(
RuntimeError: function '_has_torch_function' already has a docstring
]
[au

In [6]:
# Create iterator #1 for small train_data

BATCH_SIZE = 64
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
def preprocess_target_label(target_field):
    for example in train_data:
        setattr(example, f"label_{target_field}", getattr(example, target_field))
        
    for example in test_data:
        setattr(example, f"label_{target_field}", getattr(example, target_field))

def generate_label_iterator(dataset, target, validation = True):
    preprocess_target_label(target)
    
    label_attr = f"label_{target}"
    if validation:
        iterators = data.BucketIterator.splits(
            (dataset, valid_data, test_data),
            batch_size = BATCH_SIZE,
            device = device,
            sort_key = lambda x: len(getattr(x, label_attr)),
            sort_within_batch = False)
        return iterators[0], iterators[1], iterators[2]
    else:
        iterators = data.BucketIterator.splits(
            (dataset, test_data),
            batch_size = BATCH_SIZE,
            device = device,
            sort_key = lambda x: len(getattr(x, label_attr)),

            sort_within_batch = False)
        return iterators[0], iterators[1]

In [7]:
# Run generate_label_iterator functions

train_iterator_IT, validation_IT, test_iterator_IT = generate_label_iterator(train_data_1000, "InformationTheory", validation = True)
train_iterator_CL, validation_CL, test_iterator_CL = generate_label_iterator(train_data_1000, "ComputationalLinguistics", validation = True)
train_iterator_CV, validation_CV, test_iterator_CV = generate_label_iterator(train_data_1000, "ComputerVision", validation = True)

In [8]:
# Define RNN

import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text):

        embedded = self.embedding(text)
        
        output, hidden = self.rnn(embedded)
        
        assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        
        return self.fc(hidden.squeeze(0))


In [9]:
# Model initialization & optimizer
def generate_model_and_optimizer(embedding_dim=100, hidden_dim=256, output_dim=1, lr=1e-3):
    INPUT_DIM = len(TEXT.vocab)

    model = RNN(INPUT_DIM, embedding_dim, hidden_dim, output_dim)

    optimizer = optim.SGD(model.parameters(), lr=lr)

    model = model.to(device)
    
    return model, optimizer

model_IT, optimizer_IT = generate_model_and_optimizer()
model_CL, optimizer_CL = generate_model_and_optimizer()
model_CV, optimizer_CV = generate_model_and_optimizer()

criterion = nn.BCEWithLogitsLoss()
criterion = criterion.to(device)

In [10]:
# Evaluation functions

def binary_accuracy(preds, y):

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

def train(model, iterator, optimizer, criterion, label_field, target_field):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:

        optimizer.zero_grad()
                
        predictions = model(getattr(batch, target_field)).squeeze(1)
        
        # Use the specific label field
        loss = criterion(predictions, getattr(batch, label_field))
        
        acc = binary_accuracy(predictions, getattr(batch, label_field))
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)


def evaluate(model, iterator, criterion, label_field, target_field):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(getattr(batch, target_field)).squeeze(1)
            
            # Use the specific label field
            loss = criterion(predictions, getattr(batch, label_field))
            
            acc = binary_accuracy(predictions, getattr(batch, label_field))

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)
    
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [11]:
# # Training Loop

# N_EPOCHS = 5
# label_names = ["InformationTheory", "ComputationalLinguistics", "ComputerVision"]
# models = [model_IT, model_CL, model_CV]
# optimizers = [optimizer_IT, optimizer_CL, optimizer_CV]
# iterators = [(train_iterator_IT, test_iterator_IT), (train_iterator_CL, test_iterator_CL), (train_iterator_CV, test_iterator_CV)]


# for idx, (label_name, model, optimizer, (train_iterator, test_iterator)) in enumerate(zip(label_names, models, optimizers, iterators)):
#     print(f"Training model for {label_name}...")
#     best_valid_loss_IT = float("inf")
#     best_valid_loss_CL = float("inf")
#     best_valid_loss_CV = float("inf")
    
#     for epoch in range(N_EPOCHS):
#         start_time = time.time()
        
        
#         if label_name == label_names[0]:
#             train_loss_IT, train_acc_IT = train(model_IT, train_iterator_IT, optimizers[idx], criterion, label_name, "title")
#             test_loss_IT, test_acc_IT = evaluate(model_IT, test_iterator_IT, criterion, label_name, "title")
#             end_time = time.time()
#             epoch_mins, epoch_secs = epoch_time(start_time, end_time)
#             if test_loss_IT < best_valid_loss_IT:
#                 best_valid_loss_IT = test_loss_IT
#                 torch.save(models[0].state_dict(), "RNN_model_IT.pt")
#             print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
#             print(f'InformationTheory Test Loss: {test_loss_IT:.3f} | Test Acc: {test_acc_IT*100:.2f}%')
#             model_IT.eval()
#             y_predict = []
#             y_test = []
#             with torch.no_grad():
#                 for batch in test_iterator_IT:
#                     predictions = model_IT(batch.title).squeeze(1)
#                     rounded_preds = torch.round(torch.sigmoid(predictions))
#                     y_predict += rounded_preds.tolist()
#                     y_test += batch.InformationTheory.tolist()
                       
#         elif label_name == label_names[1]:
#             train_loss_CL, train_acc_CL = train(model_CL, train_iterator_CL, optimizers[idx], criterion, label_name, "title")
#             test_loss_CL, test_acc_CL = evaluate(model_CL, test_iterator_CL, criterion, label_name, "title")
#             end_time = time.time()
#             epoch_mins, epoch_secs = epoch_time(start_time, end_time)
#             if test_loss_CL < best_valid_loss_CL:
#                 best_valid_loss_CL = test_loss_CL
#                 torch.save(models[1].state_dict(), "RNN_model_CL.pt")
#             print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
#             print(f'ComputationalLinguistics Test Loss: {test_loss_CL:.3f} | Test Acc: {test_acc_CL*100:.2f}%')

           
#         elif label_name == label_names[2]:
#             train_loss_CV, train_acc_CV = train(model_CV, train_iterator_CV, optimizers[2], criterion, label_name, "title")
#             test_loss_CV, test_acc_CV = evaluate(model_CV, test_iterator_CV, criterion, label_name, "title")
#             end_time = time.time()
#             epoch_mins, epoch_secs = epoch_time(start_time, end_time)
#             if test_loss_CV < best_valid_loss_CV:
#                 best_valid_loss_CV = test_loss_CV
#                 torch.save(models[2].state_dict(), "RNN_model_CV.pt")
#             print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
#             print(f'ComputerVision Test Loss: {test_loss_CV:.3f} | Test Acc: {test_acc_CV*100:.2f}%')


In [12]:
# # Training Loop

# N_EPOCHS = 5
# label_names = ["InformationTheory", "ComputationalLinguistics", "ComputerVision"]
# models = [model_IT, model_CL, model_CV]
# optimizers = [optimizer_IT, optimizer_CL, optimizer_CV]
# iterators = [(train_iterator_IT, test_iterator_IT), (train_iterator_CL, test_iterator_CL), (train_iterator_CV, test_iterator_CV)]

# best_valid_losses = [float("inf"), float("inf"), float("inf")]
# model_file_names = ["RNN_model_IT.pt", "RNN_model_CL.pt", "RNN_model_CV.pt"]
# target_field = "title"

# for idx, (label_name, model, optimizer, (train_iterator, test_iterator)) in enumerate(zip(label_names, models, optimizers, iterators)):
#     print(f"Training model for {label_name}...")
    
#     for epoch in range(N_EPOCHS):
#         start_time = time.time()

#         train_loss, train_acc = train(model, train_iterator, optimizer, criterion, label_name, target_field)
#         test_loss, test_acc = evaluate(model, test_iterator, criterion, label_name, target_field)
        
#         end_time = time.time()
#         epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        
#         if test_loss < best_valid_losses[idx]:
#             best_valid_losses[idx] = test_loss
#             torch.save(model.state_dict(), model_file_names[idx])

#         print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
#         print(f'{label_name} Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')
        
#         # For predictions and ground truth collection
#         model.eval()
#         y_predict = []
#         y_test = []
#         with torch.no_grad():
#             for batch in test_iterator:
#                 predictions = model(getattr(batch, target_field)).squeeze(1)
#                 rounded_preds = torch.round(torch.sigmoid(predictions))
#                 y_predict += rounded_preds.tolist()
#                 y_test += getattr(batch, label_name).tolist()


In [13]:
def train_loop(ModelsList, OptimizersList, IteratorsList, N_EPOCHS, label_names, targe_field, modelFileNames, bestLossesList, preprocess, train_size):

    for idx, (label_name, model, optimizer, (train_iterator, test_iterator)) in enumerate(zip(label_names, ModelsList, OptimizersList, IteratorsList)):
        print(f"Training model for {preprocess}_{label_name}, using {train_size} data of {target_field}...")

        for epoch in range(N_EPOCHS):
            start_time = time.time()

            train_loss, train_acc = train(model, train_iterator, optimizer, criterion, label_name, target_field)
            test_loss, test_acc = evaluate(model, test_iterator, criterion, label_name, target_field)

            end_time = time.time()
            epoch_mins, epoch_secs = epoch_time(start_time, end_time)

            if test_loss < best_valid_losses[idx]:
                bestLossesList[idx] = test_loss
                torch.save(model.state_dict(), modelFileNames[idx])

            print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
            print(f'{label_name} Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

            # For predictions and ground truth collection
            model.eval()
            y_predict = []
            y_test = []
            with torch.no_grad():
                for batch in test_iterator:
                    predictions = model(getattr(batch, target_field)).squeeze(1)
                    rounded_preds = torch.round(torch.sigmoid(predictions))
                    y_predict += rounded_preds.tolist()
                    y_test += getattr(batch, label_name).tolist()
#     return y_predict, y_test

N_EPOCHS = 5
label_names = ["InformationTheory", "ComputationalLinguistics", "ComputerVision"]
models = [model_IT, model_CL, model_CV]
optimizers = [optimizer_IT, optimizer_CL, optimizer_CV]
iterators = [(train_iterator_IT, test_iterator_IT), (train_iterator_CL, test_iterator_CL), (train_iterator_CV, test_iterator_CV)]

best_valid_losses = [float("inf"), float("inf"), float("inf")]
model_file_names = ["RNN_model_IT.pt", "RNN_model_CL.pt", "RNN_model_CV.pt"]
target_field = "title"

# train_loop for "title", using top 1000 records of train data and preprocessing method 1
train_loop(models, optimizers, iterators, N_EPOCHS, label_names, target_field, model_file_names, best_valid_losses, "P1", "1000")




Training model for P1_InformationTheory, using 1000 data of title...
Epoch: 01 | Epoch Time: 0m 1s
InformationTheory Test Loss: 0.694 | Test Acc: 53.08%
Epoch: 02 | Epoch Time: 0m 1s
InformationTheory Test Loss: 0.710 | Test Acc: 53.10%
Epoch: 03 | Epoch Time: 0m 1s
InformationTheory Test Loss: 0.728 | Test Acc: 53.11%
Epoch: 04 | Epoch Time: 0m 1s
InformationTheory Test Loss: 0.744 | Test Acc: 53.12%
Epoch: 05 | Epoch Time: 0m 1s
InformationTheory Test Loss: 0.756 | Test Acc: 53.15%
Training model for P1_ComputationalLinguistics, using 1000 data of title...
Epoch: 01 | Epoch Time: 0m 1s
ComputationalLinguistics Test Loss: 0.577 | Test Acc: 80.35%
Epoch: 02 | Epoch Time: 0m 1s
ComputationalLinguistics Test Loss: 0.546 | Test Acc: 80.43%
Epoch: 03 | Epoch Time: 0m 1s
ComputationalLinguistics Test Loss: 0.528 | Test Acc: 80.50%
Epoch: 04 | Epoch Time: 0m 1s
ComputationalLinguistics Test Loss: 0.517 | Test Acc: 80.57%
Epoch: 05 | Epoch Time: 0m 1s
ComputationalLinguistics Test Loss: 0.510

In [14]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix, matthews_corrcoef
import numpy as np

def evaluate_model(model, iterator, label_field):
    y_predict = []
    y_test = []
    
    model.eval()
    with torch.no_grad():
        for batch in iterator:
            predictions = model(batch.title).squeeze(1)
            rounded_preds = torch.round(torch.sigmoid(predictions))
            y_predict += rounded_preds.tolist()
            y_test += getattr(batch, label_field).tolist()

    y_predict = np.asarray(y_predict)
    y_test = np.asarray(y_test)

    # Compute metrics
    recall = recall_score(y_test, y_predict, average='macro')
    precision = precision_score(y_test, y_predict, average='macro')
    f1score = f1_score(y_test, y_predict, average='macro')
    accuracy = accuracy_score(y_test, y_predict)
    matthews = matthews_corrcoef(y_test, y_predict)

    # Print metrics
    print(f"{label_field}:")
    print(confusion_matrix(y_test, y_predict))
    print('Accuracy:', accuracy)
    print('Macro Precision:', precision)
    print('Macro Recall:', recall)
    print('Macro F1 score:', f1score)
    print('MCC:', matthews)
    print("\n")

# Evaluate models
evaluate_model(model_IT, test_iterator_IT, "InformationTheory")
evaluate_model(model_CL, test_iterator_CL, "ComputationalLinguistics")
evaluate_model(model_CV, test_iterator_CV, "ComputerVision")

InformationTheory:
[[9517   70]
 [8393   86]]
Accuracy: 0.5315509797409499
Macro Precision: 0.5413305845466649
Macro Recall: 0.5014205756598811
Macro F1 score: 0.3560699520644659
MCC: 0.015324910755452786


ComputationalLinguistics:
[[14545   153]
 [ 3340    28]]
Accuracy: 0.8066533820436178
Macro Precision: 0.4839737302625584
Macro Recall: 0.4989519798288843
Macro F1 score: 0.45428797497951073
MCC: -0.008196549018356652


ComputerVision:
[[11639   208]
 [ 6101   118]]
Accuracy: 0.6507804716041182
Macro Precision: 0.5090255635249445
Macro Recall: 0.5007084620599409
Macro F1 score: 0.4114112584752512
MCC: 0.005057378501559664




In [15]:
TEXT_P2 = Field(sequential=True, tokenize="spacy", lower=True)
LABEL_P2 = LabelField(dtype=torch.float)

# Field - P2
train_datafield_P2 = [("title", TEXT_P2),
                      ("abstract", None),
                      ("InformationTheory", LABEL_P2),
                      ("ComputationalLinguistics", LABEL_P2),
                      ("ComputerVision", LABEL_P2)
                      ]

train_data_P2, test_data_P2 = TabularDataset.splits(
    path="./",
    train="train.csv", test="test.csv", format="csv",
    skip_header=True, fields=train_datafield_P2)

train_data_1000_P2, valid_data_P2 = split_dataset(train_data_P2, 1000)

# Building vocab - P2
MAX_VOCAB_SIZE = 1000

TEXT_P2.build_vocab(train_data_1000_P2, max_size=MAX_VOCAB_SIZE)
LABEL_P2.build_vocab(train_data_1000_P2)

# Create iterator #2 for P2 train_data

train_iterator_IT_P2, validation_IT_P2, test_iterator_IT_P2 = generate_label_iterator(train_data_1000_P2, "InformationTheory", validation=True)
train_iterator_CL_P2, validation_CL_P2, test_iterator_CL_P2 = generate_label_iterator(train_data_1000_P2, "ComputationalLinguistics", validation=True)
train_iterator_CV_P2, validation_CV_P2, test_iterator_CV_P2 = generate_label_iterator(train_data_1000_P2, "ComputerVision", validation=True)

model_IT_P2, optimizer_IT_P2 = generate_model_and_optimizer()
model_CL_P2, optimizer_CL_P2 = generate_model_and_optimizer()
model_CV_P2, optimizer_CV_P2 = generate_model_and_optimizer()

# Training Loop - P2

models_P2 = [model_IT_P2, model_CL_P2, model_CV_P2]
optimizers_P2 = [optimizer_IT_P2, optimizer_CL_P2, optimizer_CV_P2]
iterators_P2 = [(train_iterator_IT_P2, test_iterator_IT_P2), (train_iterator_CL_P2, test_iterator_CL_P2), (train_iterator_CV_P2, test_iterator_CV_P2)]

best_valid_losses_P2 = [float("inf"), float("inf"), float("inf")]
model_file_names_P2 = ["RNN_model_IT_P2.pt", "RNN_model_CL_P2.pt", "RNN_model_CV_P2.pt"]

train_loop(models_P2, optimizers_P2, iterators_P2, N_EPOCHS, label_names, target_field, model_file_names_P2, best_valid_losses_P2, "P2", "1000")


C:\Users\Sunny\anaconda3\lib\site-packages\torchtext\data\utils.py:123: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(f'Spacy model "{language}" could not be loaded, trying "{OLD_MODEL_SHORTCUTS[language]}" instead')


Training model for P2_InformationTheory, using 1000 data of title...
Epoch: 01 | Epoch Time: 0m 1s
InformationTheory Test Loss: 0.701 | Test Acc: 53.02%
Epoch: 02 | Epoch Time: 0m 1s
InformationTheory Test Loss: 0.715 | Test Acc: 53.02%
Epoch: 03 | Epoch Time: 0m 1s
InformationTheory Test Loss: 0.729 | Test Acc: 53.02%
Epoch: 04 | Epoch Time: 0m 2s
InformationTheory Test Loss: 0.740 | Test Acc: 53.02%
Epoch: 05 | Epoch Time: 0m 1s
InformationTheory Test Loss: 0.751 | Test Acc: 53.02%
Training model for P2_ComputationalLinguistics, using 1000 data of title...
Epoch: 01 | Epoch Time: 0m 1s
ComputationalLinguistics Test Loss: 0.711 | Test Acc: 19.22%
Epoch: 02 | Epoch Time: 0m 1s
ComputationalLinguistics Test Loss: 0.631 | Test Acc: 79.90%
Epoch: 03 | Epoch Time: 0m 1s
ComputationalLinguistics Test Loss: 0.583 | Test Acc: 80.36%
Epoch: 04 | Epoch Time: 0m 2s
ComputationalLinguistics Test Loss: 0.554 | Test Acc: 80.49%
Epoch: 05 | Epoch Time: 0m 1s
ComputationalLinguistics Test Loss: 0.536

In [16]:

# Evaluate models
evaluate_model(model_IT_P2, test_iterator_IT_P2, "InformationTheory")
evaluate_model(model_CL_P2, test_iterator_CL_P2, "ComputationalLinguistics")
evaluate_model(model_CV_P2, test_iterator_CV_P2, "ComputerVision")

InformationTheory:
[[9456  131]
 [8357  122]]
Accuracy: 0.530167164840031
Macro Precision: 0.5065308478131063
Macro Recall: 0.5003620760427228
Macro F1 score: 0.3590810877684564
MCC: 0.0030754925015638412


ComputationalLinguistics:
[[14511   187]
 [ 3334    34]]
Accuracy: 0.8051035093545887
Macro Precision: 0.4835075543677393
Macro Recall: 0.4986860962226347
Macro F1 score: 0.45537573550692434
MCC: -0.009310099164717622


ComputerVision:
[[  160 11687]
 [   82  6137]]
Accuracy: 0.3485552972434407
Macro Precision: 0.5027340330430138
Macro Recall: 0.5001600646219684
Macro F1 score: 0.26848618108041583
MCC: 0.0013230600371548133




In [17]:
# Using all whole dataset

In [18]:
TEXT = data.Field(sequential=True, tokenize=custom_tokenizer, lower=False)
LABEL = LabelField(dtype = torch.float)
 
train_datafield = [("title", TEXT), 
                   ("abstract", None),
                   ("InformationTheory", LABEL), 
                   ("ComputationalLinguistics", LABEL),
                   ("ComputerVision", LABEL)
                   ]

train_data, test_data = TabularDataset.splits(
    path = "./",
    train = "train.csv", test = "test.csv", format = "csv",
    skip_header = True, fields = train_datafield)

MAX_VOCAB_SIZE = 1000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

# Run generate_label_iterator functions

train_iterator_IT, test_iterator_IT = generate_label_iterator(train_data, "InformationTheory", validation = False)
train_iterator_CL, test_iterator_CL = generate_label_iterator(train_data, "ComputationalLinguistics", validation = False)
train_iterator_CV, test_iterator_CV = generate_label_iterator(train_data, "ComputerVision", validation = False)

model_IT, optimizer_IT = generate_model_and_optimizer()
model_CL, optimizer_CL = generate_model_and_optimizer()
model_CV, optimizer_CV = generate_model_and_optimizer()

criterion = nn.BCEWithLogitsLoss()
criterion = criterion.to(device)

In [19]:
# Training Loop

models = [model_IT, model_CL, model_CV]
optimizers = [optimizer_IT, optimizer_CL, optimizer_CV]
iterators = [(train_iterator_IT, test_iterator_IT), (train_iterator_CL, test_iterator_CL), (train_iterator_CV, test_iterator_CV)]


best_valid_losses = [float("inf"), float("inf"), float("inf")]
model_file_names = ["RNN_model_IT.pt", "RNN_model_CL.pt", "RNN_model_CV.pt"]

train_loop(models, optimizers, iterators, N_EPOCHS, label_names, target_field, model_file_names, best_valid_losses, "P1", "ALL")

Training model for P1_InformationTheory, using ALL data of title...
Epoch: 01 | Epoch Time: 0m 34s
InformationTheory Test Loss: 0.804 | Test Acc: 53.06%
Epoch: 02 | Epoch Time: 0m 32s
InformationTheory Test Loss: 0.819 | Test Acc: 53.10%
Epoch: 03 | Epoch Time: 0m 32s
InformationTheory Test Loss: 0.808 | Test Acc: 53.10%
Epoch: 04 | Epoch Time: 0m 32s
InformationTheory Test Loss: 0.813 | Test Acc: 53.11%
Epoch: 05 | Epoch Time: 0m 32s
InformationTheory Test Loss: 0.805 | Test Acc: 53.11%
Training model for P1_ComputationalLinguistics, using ALL data of title...
Epoch: 01 | Epoch Time: 0m 31s
ComputationalLinguistics Test Loss: 0.494 | Test Acc: 81.02%
Epoch: 02 | Epoch Time: 0m 32s
ComputationalLinguistics Test Loss: 0.492 | Test Acc: 81.09%
Epoch: 03 | Epoch Time: 0m 31s
ComputationalLinguistics Test Loss: 0.492 | Test Acc: 81.19%
Epoch: 04 | Epoch Time: 0m 31s
ComputationalLinguistics Test Loss: 0.492 | Test Acc: 81.23%
Epoch: 05 | Epoch Time: 0m 32s
ComputationalLinguistics Test Los

In [ ]:
# # Training Loop

# N_EPOCHS = 5
# label_names = ["InformationTheory", "ComputationalLinguistics", "ComputerVision"]
# models = [model_IT, model_CL, model_CV]
# optimizers = [optimizer_IT, optimizer_CL, optimizer_CV]
# iterators = [(train_iterator_IT, test_iterator_IT), (train_iterator_CL, test_iterator_CL), (train_iterator_CV, test_iterator_CV)]


# for idx, (label_name, model, optimizer, (train_iterator, test_iterator)) in enumerate(zip(label_names, models, optimizers, iterators)):
#     print(f"Training model for {label_name}...")
#     best_valid_loss_IT = float("inf")
#     best_valid_loss_CL = float("inf")
#     best_valid_loss_CV = float("inf")
    
#     for epoch in range(N_EPOCHS):
#         start_time = time.time()
        
        
#         if label_name == label_names[0]:
#             train_loss_IT, train_acc_IT = train(model_IT, train_iterator_IT, optimizers[idx], criterion, label_name, "title")
#             test_loss_IT, test_acc_IT = evaluate(model_IT, test_iterator_IT, criterion, label_name, "title")
#             end_time = time.time()
#             epoch_mins, epoch_secs = epoch_time(start_time, end_time)
#             if test_loss_IT < best_valid_loss_IT:
#                 best_valid_loss_IT = test_loss_IT
#                 torch.save(models[0].state_dict(), "RNN_model_IT.pt")
#             print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
#             print(f'InformationTheory Test Loss: {test_loss_IT:.3f} | Test Acc: {test_acc_IT*100:.2f}%')
#             model_IT.eval()
#             y_predict = []
#             y_test = []
#             with torch.no_grad():
#                 for batch in test_iterator_IT:
#                     predictions = model_IT(batch.title).squeeze(1)
#                     rounded_preds = torch.round(torch.sigmoid(predictions))
#                     y_predict += rounded_preds.tolist()
#                     y_test += batch.InformationTheory.tolist()
                       
#         elif label_name == label_names[1]:
#             train_loss_CL, train_acc_CL = train(model_CL, train_iterator_CL, optimizers[idx], criterion, label_name, "title")
#             test_loss_CL, test_acc_CL = evaluate(model_CL, test_iterator_CL, criterion, label_name, "title")
#             end_time = time.time()
#             epoch_mins, epoch_secs = epoch_time(start_time, end_time)
#             if test_loss_CL < best_valid_loss_CL:
#                 best_valid_loss_CL = test_loss_CL
#                 torch.save(models[1].state_dict(), "RNN_model_CL.pt")
#             print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
#             print(f'ComputationalLinguistics Test Loss: {test_loss_CL:.3f} | Test Acc: {test_acc_CL*100:.2f}%')

           
#         elif label_name == label_names[2]:
#             train_loss_CV, train_acc_CV = train(model_CV, train_iterator_CV, optimizers[2], criterion, label_name, "title")
#             test_loss_CV, test_acc_CV = evaluate(model_CV, test_iterator_CV, criterion, label_name, "title")
#             end_time = time.time()
#             epoch_mins, epoch_secs = epoch_time(start_time, end_time)
#             if test_loss_CV < best_valid_loss_CV:
#                 best_valid_loss_CV = test_loss_CV
#                 torch.save(models[2].state_dict(), "RNN_model_CV.pt")
#             print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
#             print(f'ComputerVision Test Loss: {test_loss_CV:.3f} | Test Acc: {test_acc_CV*100:.2f}%')


In [20]:
TEXT_P2 = Field(sequential=True, tokenize="spacy", lower=True)
LABEL_P2 = LabelField(dtype=torch.float)

# Field - P2
train_datafield_P2 = [("title", TEXT_P2),
                      ("abstract", None),
                      ("InformationTheory", LABEL_P2),
                      ("ComputationalLinguistics", LABEL_P2),
                      ("ComputerVision", LABEL_P2)
                      ]

train_data_P2, test_data_P2 = TabularDataset.splits(
    path="./",
    train="train.csv", test="test.csv", format="csv",
    skip_header=True, fields=train_datafield_P2)


# Building vocab - P2
MAX_VOCAB_SIZE = 1000

TEXT_P2.build_vocab(train_data_P2, max_size=MAX_VOCAB_SIZE)
LABEL_P2.build_vocab(train_data_P2)

# Create iterator #2 for P2 train_data

train_iterator_IT_P2, test_iterator_IT_P2 = generate_label_iterator(train_data_1000_P2, "InformationTheory", validation=False)
train_iterator_CL_P2, test_iterator_CL_P2 = generate_label_iterator(train_data_1000_P2, "ComputationalLinguistics", validation=False)
train_iterator_CV_P2, test_iterator_CV_P2 = generate_label_iterator(train_data_1000_P2, "ComputerVision", validation=False)

model_IT_P2, optimizer_IT_P2 = generate_model_and_optimizer()
model_CL_P2, optimizer_CL_P2 = generate_model_and_optimizer()
model_CV_P2, optimizer_CV_P2 = generate_model_and_optimizer()

# Training Loop - P2

models_P2 = [model_IT_P2, model_CL_P2, model_CV_P2]
optimizers_P2 = [optimizer_IT_P2, optimizer_CL_P2, optimizer_CV_P2]
iterators_P2 = [(train_iterator_IT_P2, test_iterator_IT_P2), (train_iterator_CL_P2, test_iterator_CL_P2), (train_iterator_CV_P2, test_iterator_CV_P2)]

best_valid_losses_P2 = [float("inf"), float("inf"), float("inf")]
model_file_names_P2 = ["RNN_model_IT_P2.pt", "RNN_model_CL_P2.pt", "RNN_model_CV_P2.pt"]

train_loop(models_P2, optimizers_P2, iterators_P2, N_EPOCHS, label_names, target_field, model_file_names_P2, best_valid_losses_P2, "P2", "ALL")

C:\Users\Sunny\anaconda3\lib\site-packages\torchtext\data\utils.py:123: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(f'Spacy model "{language}" could not be loaded, trying "{OLD_MODEL_SHORTCUTS[language]}" instead')


Training model for P2_InformationTheory, using ALL data of title...
Epoch: 01 | Epoch Time: 0m 1s
InformationTheory Test Loss: 0.693 | Test Acc: 53.02%
Epoch: 02 | Epoch Time: 0m 1s
InformationTheory Test Loss: 0.705 | Test Acc: 53.02%
Epoch: 03 | Epoch Time: 0m 1s
InformationTheory Test Loss: 0.720 | Test Acc: 53.00%
Epoch: 04 | Epoch Time: 0m 1s
InformationTheory Test Loss: 0.733 | Test Acc: 53.01%
Epoch: 05 | Epoch Time: 0m 1s
InformationTheory Test Loss: 0.746 | Test Acc: 53.02%
Training model for P2_ComputationalLinguistics, using ALL data of title...
Epoch: 01 | Epoch Time: 0m 1s
ComputationalLinguistics Test Loss: 0.602 | Test Acc: 80.35%
Epoch: 02 | Epoch Time: 0m 1s
ComputationalLinguistics Test Loss: 0.560 | Test Acc: 80.39%
Epoch: 03 | Epoch Time: 0m 1s
ComputationalLinguistics Test Loss: 0.536 | Test Acc: 80.40%
Epoch: 04 | Epoch Time: 0m 1s
ComputationalLinguistics Test Loss: 0.522 | Test Acc: 80.42%
Epoch: 05 | Epoch Time: 0m 1s
ComputationalLinguistics Test Loss: 0.513 |

In [ ]:
### For ABSTRACT USINIG 1000 DATA

In [30]:
TEXT = data.Field(sequential=True, tokenize=custom_tokenizer, lower=False)
LABEL = LabelField(dtype = torch.float)
 
train_datafield = [("title", None), 
                   ("abstract", TEXT),
                   ("InformationTheory", LABEL), 
                   ("ComputationalLinguistics", LABEL),
                   ("ComputerVision", LABEL)
                   ]

train_data, test_data = TabularDataset.splits(
    path = "./",
    train = "train.csv", test = "test.csv", format = "csv",
    skip_header = True, fields = train_datafield)

train_data_1000, valid_data = split_dataset(train_data, 1000)
MAX_VOCAB_SIZE = 10000

TEXT.build_vocab(train_data_1000, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data_1000)

# Run generate_label_iterator functions

train_iterator_IT, validation_IT, test_iterator_IT = generate_label_iterator(train_data_1000, "InformationTheory", validation = True)
train_iterator_CL, validation_CL, test_iterator_CL = generate_label_iterator(train_data_1000, "ComputationalLinguistics", validation = True)
train_iterator_CV, validation_CV, test_iterator_CV = generate_label_iterator(train_data_1000, "ComputerVision", validation = True)

model_IT, optimizer_IT = generate_model_and_optimizer()
model_CL, optimizer_CL = generate_model_and_optimizer()
model_CV, optimizer_CV = generate_model_and_optimizer()

In [31]:
# Training Loop

models = [model_IT, model_CL, model_CV]
optimizers = [optimizer_IT, optimizer_CL, optimizer_CV]
iterators = [(train_iterator_IT, test_iterator_IT), (train_iterator_CL, test_iterator_CL), (train_iterator_CV, test_iterator_CV)]
target_field = "abstract"

best_valid_losses = [float("inf"), float("inf"), float("inf")]
model_file_names = ["RNN_model_IT.pt", "RNN_model_CL.pt", "RNN_model_CV.pt"]

train_loop(models, optimizers, iterators, N_EPOCHS, label_names, target_field, model_file_names, best_valid_losses, "P1", "1000")

Training model for P1_InformationTheory, using 1000 data of abstract...
Epoch: 01 | Epoch Time: 0m 15s
InformationTheory Test Loss: 0.692 | Test Acc: 53.02%
Epoch: 02 | Epoch Time: 0m 16s
InformationTheory Test Loss: 0.699 | Test Acc: 53.03%
Epoch: 03 | Epoch Time: 0m 17s
InformationTheory Test Loss: 0.710 | Test Acc: 53.00%
Epoch: 04 | Epoch Time: 0m 17s
InformationTheory Test Loss: 0.723 | Test Acc: 53.00%


KeyboardInterrupt: 

In [ ]:
TEXT = data.Field(sequential=True, tokenize=custom_tokenizer, lower=False)
LABEL = LabelField(dtype = torch.float)
 
train_datafield = [("title", None), 
                   ("abstract", TEXT),
                   ("InformationTheory", LABEL), 
                   ("ComputationalLinguistics", LABEL),
                   ("ComputerVision", LABEL)
                   ]

train_data, test_data = TabularDataset.splits(
    path = "./",
    train = "train.csv", test = "test.csv", format = "csv",
    skip_header = True, fields = train_datafield)

train_data_1000, valid_data = split_dataset(train_data, 1000)
MAX_VOCAB_SIZE = 1000

TEXT.build_vocab(train_data_1000, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data_1000)

# Run generate_label_iterator functions

train_iterator_IT, validation_IT, test_iterator_IT = generate_label_iterator(train_data_1000, "InformationTheory", validation = True)
train_iterator_CL, validation_CL, test_iterator_CL = generate_label_iterator(train_data_1000, "ComputationalLinguistics", validation = True)
train_iterator_CV, validation_CV, test_iterator_CV = generate_label_iterator(train_data_1000, "ComputerVision", validation = True)

model_IT, optimizer_IT = generate_model_and_optimizer()
model_CL, optimizer_CL = generate_model_and_optimizer()
model_CV, optimizer_CV = generate_model_and_optimizer()

In [23]:
# Training Loop

models = [model_IT, model_CL, model_CV]
optimizers = [optimizer_IT, optimizer_CL, optimizer_CV]
iterators = [(train_iterator_IT, test_iterator_IT), (train_iterator_CL, test_iterator_CL), (train_iterator_CV, test_iterator_CV)]
target_field = "abstract"

best_valid_losses = [float("inf"), float("inf"), float("inf")]
model_file_names = ["RNN_model_IT.pt", "RNN_model_CL.pt", "RNN_model_CV.pt"]

train_loop(models, optimizers, iterators, N_EPOCHS, label_names, target_field, model_file_names, best_valid_losses, "P1", "1000")

Training model for P1_InformationTheory, using 1000 data of abstract...
Epoch: 01 | Epoch Time: 0m 21s
InformationTheory Test Loss: 0.691 | Test Acc: 53.08%
Epoch: 02 | Epoch Time: 0m 19s
InformationTheory Test Loss: 0.696 | Test Acc: 53.12%
Epoch: 03 | Epoch Time: 0m 18s
InformationTheory Test Loss: 0.706 | Test Acc: 53.15%
Epoch: 04 | Epoch Time: 0m 18s
InformationTheory Test Loss: 0.719 | Test Acc: 53.15%
Epoch: 05 | Epoch Time: 0m 18s
InformationTheory Test Loss: 0.731 | Test Acc: 53.15%
Training model for P1_ComputationalLinguistics, using 1000 data of abstract...
Epoch: 01 | Epoch Time: 0m 18s
ComputationalLinguistics Test Loss: 0.654 | Test Acc: 80.13%
Epoch: 02 | Epoch Time: 0m 18s
ComputationalLinguistics Test Loss: 0.594 | Test Acc: 80.16%
Epoch: 03 | Epoch Time: 0m 18s
ComputationalLinguistics Test Loss: 0.559 | Test Acc: 80.19%
Epoch: 04 | Epoch Time: 0m 18s
ComputationalLinguistics Test Loss: 0.538 | Test Acc: 80.20%
Epoch: 05 | Epoch Time: 0m 18s
ComputationalLinguistics 

In [24]:
TEXT = data.Field(sequential=True, tokenize="spacy", lower=True)
LABEL = LabelField(dtype = torch.float)
 
train_datafield = [("title", None), 
                    ("abstract", TEXT),
                   ("InformationTheory", LABEL), 
                   ("ComputationalLinguistics", LABEL),
                   ("ComputerVision", LABEL)
                   ]

train_data, test_data = TabularDataset.splits(
    path = "./",
    train = "train.csv", test = "test.csv", format = "csv",
    skip_header = True, fields = train_datafield)

train_data_1000, valid_data = split_dataset(train_data, 1000)
MAX_VOCAB_SIZE = 1000

TEXT.build_vocab(train_data_1000, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data_1000)

# Run generate_label_iterator functions

train_iterator_IT_P2, validation_IT_P2, test_iterator_IT_P2 = generate_label_iterator(train_data_1000, "InformationTheory", validation = True)
train_iterator_CL_P2, validation_CL_P2, test_iterator_CL_P2 = generate_label_iterator(train_data_1000, "ComputationalLinguistics", validation = True)
train_iterator_CV_P2, validation_CV_P2, test_iterator_CV_P2 = generate_label_iterator(train_data_1000, "ComputerVision", validation = True)

model_IT_P2, optimizer_IT_P2 = generate_model_and_optimizer()
model_CL_P2, optimizer_CL_P2 = generate_model_and_optimizer()
model_CV_P2, optimizer_CV_P2 = generate_model_and_optimizer()

# Training Loop

models_P2 = [model_IT_P2, model_CL_P2, model_CV]
optimizers_P2 = [optimizer_IT, optimizer_CL, optimizer_CV]
iterators_P2 = [(train_iterator_IT, test_iterator_IT), (train_iterator_CL, test_iterator_CL), (train_iterator_CV, test_iterator_CV)]


best_valid_losses = [float("inf"), float("inf"), float("inf")]
model_file_names = ["RNN_model_IT_P2.pt", "RNN_model_CL_P2.pt", "RNN_model_CV_P2.pt"]

train_loop(models_P2, optimizers_P2, iterators_P2, N_EPOCHS, label_names, target_field, model_file_names_P2, best_valid_losses_P2, "P2", "1000")

Training model for P2_InformationTheory, using 1000 data of abstract...
Epoch: 01 | Epoch Time: 0m 19s
InformationTheory Test Loss: 0.692 | Test Acc: 53.07%
Epoch: 02 | Epoch Time: 0m 19s
InformationTheory Test Loss: 0.692 | Test Acc: 53.07%
Epoch: 03 | Epoch Time: 0m 19s
InformationTheory Test Loss: 0.692 | Test Acc: 53.07%
Epoch: 04 | Epoch Time: 0m 18s
InformationTheory Test Loss: 0.692 | Test Acc: 53.07%
Epoch: 05 | Epoch Time: 0m 18s
InformationTheory Test Loss: 0.692 | Test Acc: 53.07%
Training model for P2_ComputationalLinguistics, using 1000 data of abstract...
Epoch: 01 | Epoch Time: 0m 18s
ComputationalLinguistics Test Loss: 0.632 | Test Acc: 80.40%
Epoch: 02 | Epoch Time: 0m 19s
ComputationalLinguistics Test Loss: 0.632 | Test Acc: 80.40%
Epoch: 03 | Epoch Time: 0m 18s
ComputationalLinguistics Test Loss: 0.632 | Test Acc: 80.40%
Epoch: 04 | Epoch Time: 0m 18s
ComputationalLinguistics Test Loss: 0.632 | Test Acc: 80.40%
Epoch: 05 | Epoch Time: 0m 18s
ComputationalLinguistics 

In [ ]:
# ABSTRACT USING ALL DATA

In [27]:
#P1
TEXT = data.Field(sequential=True, tokenize=custom_tokenizer, lower=False)
LABEL = LabelField(dtype = torch.float)

train_datafield = [("title", None), 
                   ("abstract", TEXT),
                   ("InformationTheory", LABEL), 
                   ("ComputationalLinguistics", LABEL),
                   ("ComputerVision", LABEL)
                   ]

train_data, test_data = TabularDataset.splits(
    path = "./",
    train = "train.csv", test = "test.csv", format = "csv",
    skip_header = True, fields = train_datafield)

MAX_VOCAB_SIZE = 1000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

# Run generate_label_iterator functions

train_iterator_IT, test_iterator_IT = generate_label_iterator(train_data, "InformationTheory", validation = False)
train_iterator_CL, test_iterator_CL = generate_label_iterator(train_data, "ComputationalLinguistics", validation = False)
train_iterator_CV, test_iterator_CV = generate_label_iterator(train_data, "ComputerVision", validation = False)

model_IT, optimizer_IT = generate_model_and_optimizer()
model_CL, optimizer_CL = generate_model_and_optimizer()
model_CV, optimizer_CV = generate_model_and_optimizer()

In [28]:
# Training Loop

models = [model_IT, model_CL, model_CV]
optimizers = [optimizer_IT, optimizer_CL, optimizer_CV]
iterators = [(train_iterator_IT, test_iterator_IT), (train_iterator_CL, test_iterator_CL), (train_iterator_CV, test_iterator_CV)]

best_valid_losses = [float("inf"), float("inf"), float("inf")]
model_file_names = ["RNN_model_IT.pt", "RNN_model_CL.pt", "RNN_model_CV.pt"]

train_loop(models, optimizers, iterators, N_EPOCHS, label_names, target_field, model_file_names, best_valid_losses, "P1", "ALL")

Training model for P1_InformationTheory, using ALL data of abstract...
Epoch: 01 | Epoch Time: 6m 25s
InformationTheory Test Loss: 0.807 | Test Acc: 53.07%
Epoch: 02 | Epoch Time: 5m 32s
InformationTheory Test Loss: 0.822 | Test Acc: 53.06%
Epoch: 03 | Epoch Time: 5m 28s
InformationTheory Test Loss: 0.810 | Test Acc: 53.06%
Epoch: 04 | Epoch Time: 5m 41s
InformationTheory Test Loss: 0.815 | Test Acc: 53.06%
Epoch: 05 | Epoch Time: 5m 58s
InformationTheory Test Loss: 0.808 | Test Acc: 53.06%
Training model for P1_ComputationalLinguistics, using ALL data of abstract...
Epoch: 01 | Epoch Time: 5m 57s
ComputationalLinguistics Test Loss: 0.491 | Test Acc: 81.38%
Epoch: 02 | Epoch Time: 5m 56s
ComputationalLinguistics Test Loss: 0.489 | Test Acc: 81.38%
Epoch: 03 | Epoch Time: 5m 57s
ComputationalLinguistics Test Loss: 0.489 | Test Acc: 81.38%
Epoch: 04 | Epoch Time: 5m 56s
ComputationalLinguistics Test Loss: 0.490 | Test Acc: 81.38%
Epoch: 05 | Epoch Time: 5m 57s
ComputationalLinguistics Te

In [29]:
#P2

TEXT = data.Field(sequential=True, tokenize="spacy", lower=True)
LABEL = LabelField(dtype = torch.float)
 
train_datafield = [("title", None), 
                    ("abstract", TEXT),
                   ("InformationTheory", LABEL), 
                   ("ComputationalLinguistics", LABEL),
                   ("ComputerVision", LABEL)
                   ]

train_data, test_data = TabularDataset.splits(
    path = "./",
    train = "train.csv", test = "test.csv", format = "csv",
    skip_header = True, fields = train_datafield)

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

# Run generate_label_iterator functions

train_iterator_IT_P2, test_iterator_IT_P2 = generate_label_iterator(train_data_1000, "InformationTheory", validation = False)
train_iterator_CL_P2, test_iterator_CL_P2 = generate_label_iterator(train_data_1000, "ComputationalLinguistics", validation = False)
train_iterator_CV_P2, test_iterator_CV_P2 = generate_label_iterator(train_data_1000, "ComputerVision", validation = False)

model_IT_P2, optimizer_IT_P2 = generate_model_and_optimizer()
model_CL_P2, optimizer_CL_P2 = generate_model_and_optimizer()
model_CV_P2, optimizer_CV_P2 = generate_model_and_optimizer()

# Training Loop

models_P2 = [model_IT_P2, model_CL_P2, model_CV]
optimizers_P2 = [optimizer_IT, optimizer_CL, optimizer_CV]
iterators_P2 = [(train_iterator_IT, test_iterator_IT), (train_iterator_CL, test_iterator_CL), (train_iterator_CV, test_iterator_CV)]


best_valid_losses = [float("inf"), float("inf"), float("inf")]
model_file_names = ["RNN_model_IT_P2.pt", "RNN_model_CL_P2.pt", "RNN_model_CV_P2.pt"]

train_loop(models_P2, optimizers_P2, iterators_P2, N_EPOCHS, label_names, target_field, model_file_names_P2, best_valid_losses_P2, "P2", "ALL")

Training model for P2_InformationTheory, using ALL data of abstract...
Epoch: 01 | Epoch Time: 5m 58s
InformationTheory Test Loss: 0.698 | Test Acc: 46.85%
Epoch: 02 | Epoch Time: 5m 59s
InformationTheory Test Loss: 0.698 | Test Acc: 46.85%
Epoch: 03 | Epoch Time: 5m 59s
InformationTheory Test Loss: 0.698 | Test Acc: 46.85%
Epoch: 04 | Epoch Time: 6m 2s
InformationTheory Test Loss: 0.698 | Test Acc: 46.85%
Epoch: 05 | Epoch Time: 6m 1s
InformationTheory Test Loss: 0.698 | Test Acc: 46.85%
Training model for P2_ComputationalLinguistics, using ALL data of abstract...
Epoch: 01 | Epoch Time: 309m 6s
ComputationalLinguistics Test Loss: 0.707 | Test Acc: 18.68%
Epoch: 02 | Epoch Time: 5m 36s
ComputationalLinguistics Test Loss: 0.707 | Test Acc: 18.68%
Epoch: 03 | Epoch Time: 5m 37s
ComputationalLinguistics Test Loss: 0.707 | Test Acc: 18.68%
Epoch: 04 | Epoch Time: 6m 0s
ComputationalLinguistics Test Loss: 0.707 | Test Acc: 18.68%
Epoch: 05 | Epoch Time: 5m 59s
ComputationalLinguistics Test

KeyboardInterrupt: 